## Install

In [1]:
#!pip install moviepy
#!pip install transformers torchaudio
#!pip install spleeter
#!pip install torchaudio
import media_embedder


## Data Loader

In [1]:
import os
import pandas as pd

# Specify the directory path to read files from
folder_path = '/Volumes/SeaGate/MoviesCMD/videos' #/Volumes/SeaGate/MoviesCMD/videos'
meta_path="/Users/markrademaker/Downloads/Work/Scriptie/Code/CondensedMovies-master/data/metadata"
data=pd.DataFrame()
# Load each CSV file into a DataFrame
casts_df = pd.read_csv(meta_path+'/casts.csv')
clips_df = pd.read_csv(meta_path+'/clips.csv')
descriptions_df = pd.read_csv(meta_path+'/descriptions.csv')
durations_df = pd.read_csv(meta_path+'/durations.csv')
movies_df = pd.read_csv(meta_path+'/movies.csv')
movie_info_df = pd.read_csv(meta_path+'/movie_info.csv')
split_df = pd.read_csv(meta_path+'/split.csv')
merged_df = pd.merge(movies_df, movie_info_df, on='imdbid', how='outer')
merged_df = pd.merge(merged_df, casts_df, on='imdbid', how='outer')
merged_df = pd.merge(merged_df, split_df, on='imdbid', how='outer')
merged_df = pd.merge(merged_df, casts_df, on='imdbid', how='outer')
video_df = pd.merge(clips_df, descriptions_df, on=['videoid','imdbid'], how='outer')
video_df = pd.merge(video_df, merged_df, on=['imdbid'], how='outer')

data_from_files = []
# Loop through each file in the specified directory
for folder in os.listdir(folder_path):
    print(f"Year {folder} movies loading..")
    #for file in os.listdir(folder_path+"/"+folder):
    # Construct full file path
    subfolder_path = os.path.join(folder_path, folder)
    if(folder.endswith("Store")):
        continue
    for file in os.listdir(subfolder_path):
        # Check if it is a file
        file_path=os.path.join(subfolder_path,file)
        if os.path.isfile(file_path) and file_path.endswith(".mkv"):
            # Open and read the file
            data_from_files.append({'videoid': file[:-4], 'path': file_path, 'audio_path': file_path[:-4] + "_audio.opus",
                                     'acc_path': file_path[:-4] + "_audio/accompaniment.wav",
                                     'voc_path': file_path[:-4] + "_audio/vocals.wav"})
files_df = pd.DataFrame(data_from_files)
# Assuming 'ID' is a common column between 'merged_df' and 'files_df'
# Update 'merged_df' by merging it with 'files_df'
# This example uses a left join to keep all rows from 'merged_df' and add matching rows from 'files_df'
print(files_df)

video_df = pd.merge(video_df, files_df, on='videoid', how='outer')
print(video_df)
print(f"Dataframe columns: {video_df.columns}")


Year 2020 movies loading..
Year 2019 movies loading..
Year 2018 movies loading..
Year 2015 movies loading..
Year 2014 movies loading..
Year 2016 movies loading..
Year 2017 movies loading..
Year 2013 movies loading..
Year 2012 movies loading..
Year 2011 movies loading..
Year videos movies loading..
           videoid                                               path  \
0      Ok63vpXNhNc  /Volumes/SeaGate/MoviesCMD/videos/2020/Ok63vpX...   
1      OfLhd_wJux8  /Volumes/SeaGate/MoviesCMD/videos/2020/OfLhd_w...   
2      TGghm3K1NXQ  /Volumes/SeaGate/MoviesCMD/videos/2020/TGghm3K...   
3      u0RqfETo2ok  /Volumes/SeaGate/MoviesCMD/videos/2020/u0RqfET...   
4      oaKjYmfK_Pw  /Volumes/SeaGate/MoviesCMD/videos/2020/oaKjYmf...   
...            ...                                                ...   
22344  Hy37hjPUFWo  /Volumes/SeaGate/MoviesCMD/videos/2011/Hy37hjP...   
22345  Nfrk2UdEOcQ  /Volumes/SeaGate/MoviesCMD/videos/2011/Nfrk2Ud...   
22346  C_SFevIz1FI  /Volumes/SeaGate/MoviesC

In [43]:
# Filter out rows where 'path' is NaN
filtered_video_df = video_df[video_df['path'].notna()]

# Now, you can group by 'imdbid' if you need to perform further operations on the grouped data
grouped_by_imdbid =filtered_video_df.groupby('imdbid')

In [7]:
print(grouped_by_imdbid.head())
audio_series=pd.Series(grouped_by_imdbid['audio_path'])
print(audio_series.loc[1][1].iloc[0])
from IPython.display import Audio
print(audio_series.loc[1][1].iloc[0])
print(audio_series.loc[1][1].iloc[2])


Empty DataFrame
Columns: [videoid, year_x, clip_idx, clip_tot, upload_year, title_x, clip_name, imdbid, description, title_y, year_y, genre, country, cast_x, split, cast_y, path]
Index: []


KeyError: 'Column not found: audio_path'

## CNN and Wav2Vec

In [2]:
import torch
import torchaudio
import torchaudio.transforms as T
import media_embedder
import cv2
import media_embedder
from media_embedder import MediaEmbedder
video_path = "/Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_0-100_no_audio.mp4"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"The video's frame rate is: {fps} FPS")

cap.release()

media_embed = MediaEmbedder()
chunk_duration=1 #seconds for synchronization
audio_embeddings = media_embed.embed_audio(
    "/Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_0-100_audio.mp3", chunk_duration)
#video_embeddings = media_embed.embed_video(
#    "/Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_0-100_no_audio.mp4", chunk_duration, fps)

The video's frame rate is: 25.0 FPS
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import subprocess
import numpy as np
import tqdm

failed_audio=[]
def separate_audio_video(input_file, output_video, output_audio):
    """
    Separates the audio and video streams of a video file using ffmpeg.

    Parameters:
    - input_file: Path to the input video file.
    - output_video: Path for the output video file (without audio).
    - output_audio: Path for the output audio file (without video).
    """
    if os.path.exists(output_audio):
        print(f"Audio file {output_audio} already exists. Skipping...")
        return
    # Extract audio (no video)
    if '/.' in input_file:
        return
    try:
        print(output_audio)
       # subprocess.run(['ffmpeg', '-v', 'error', '-i', str(input_file), '-vn', '-c:a', 'copy', str(output_audio)], check=True)
    except subprocess.CalledProcessError as e:
        with open('failed_audio.out', 'a') as fid:  # 'a' opens the file in append mode
            fid.write(input_file + '\n')
    # Extract video (no audio)
    #subprocess.run(['ffmpeg', '-v', 'error', '-i', str(input_file), '-an', '-c:v', 'copy', str(output_video)], check=True)



# Example usage
for index, row in tqdm.tqdm(video_df.iterrows()):  # Adjust extension based on codec, e.g., .ogg, .opus
    input_webm_file = row['path']  # Directly use row access for the 'path'
    if(len(str(input_webm_file))<4) or str(input_webm_file).endswith('.srt') :
        continue
    # Extract directory path and base filename without extension
    directory, filename_with_extension = os.path.split(str(input_webm_file))
    filename_without_extension, _ = os.path.splitext(filename_with_extension)
    
    # Define output paths (adjust these paths as needed)
    output_directory = directory  # This could be modified if you want to save outputs elsewhere
    output_video_file = os.path.join(output_directory, filename_without_extension + "_video.webm")
    output_audio_file = os.path.join(output_directory, filename_without_extension + "_audio.opus")  # Or .ogg, .mp3, etc.
    
    # Call the function with the file paths
    #separate_audio_video(input_webm_file, output_video_file, output_audio_file)

video_df.loc[~video_df['path'].isna(), 'audio_path'] = video_df['path'].str[:-4] + "_audio.opus"

# Assuming you want to similarly update 'video_path' only if it's NaN
video_df.loc[~video_df['path'].isna(), 'video_path'] = video_df['path'].str[:-4] + "_video.webm"


22349it [00:01, 11735.88it/s]


In [5]:
print(video_df['audio_path'])

0                                                      NaN
1                                                      NaN
2                                                      NaN
3                                                      NaN
4                                                      NaN
                               ...                        
59385    /Volumes/SeaGate/MoviesCMD/videos/2011/Hy37hjP...
59386    /Volumes/SeaGate/MoviesCMD/videos/2011/Nfrk2Ud...
59387    /Volumes/SeaGate/MoviesCMD/videos/2011/C_SFevI...
59388    /Volumes/SeaGate/MoviesCMD/videos/2011/wAjHfBk...
59389    /Volumes/SeaGate/MoviesCMD/videos/2011/lt9IJX0...
Name: audio_path, Length: 59390, dtype: object


## Spleeter

In [2]:
!spleeter separate -h
video_df=video_df.dropna(subset=['audio_path'])
import tqdm
for index, row in video_df.iterrows():
    loc = row['audio_path']
    output_dir= os.path.dirname(str(loc))
    # Check if the directory already exists
    if not os.path.exists(loc[:-5]):
        # If the directory does not exist, create it
        # Run the spleeter command
        !spleeter separate -o {output_dir} {loc}


Usage: spleeter separate [OPTIONS] FILES...
Try 'spleeter separate --help' for help.

Error: no such option: -h
/Volumes/SeaGate/MoviesCMD/videos/2019/Sv-BxH3SVS8_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/IsZdfna1LKA_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/88T3elu2wfE_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/ojoC-Kbzpo8_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/deUgUoJ4z5I_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/ocDL_b6BRE4_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/0BM-Q3BDrkw_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/9jfRE_FljrE_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/iLN9GLRC5is_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/ulJXiB5i_q0_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/tdzX5AKWiDw_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/mycAsRhAr_M_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/WgXQnXPb-TA_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/WgMhaDEPGXo_audio
/Volumes/SeaGate/MoviesCMD/videos/2019/HummNgSGn8k_audio
/Volumes/SeaGate/MoviesCMD/videos

In [5]:
import os
import subprocess
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import os

max_workers = os.cpu_count() 
print(max_workers)
# Function to run Spleeter on a single file
def run_spleeter(loc):
    output_dir = os.path.dirname(str(loc))
    spleeter_command = ["spleeter", "separate", "-o", output_dir, loc]
    if not os.path.exists(loc[:-5]):
        try:
            subprocess.run(spleeter_command, check=True)
            print(f"Processing {loc} completed.")
        except subprocess.CalledProcessError as e:
            print(f"Error processing {loc}: {e}")
    else:
        print(f"Directory {output_dir} already exists, skipping...")

# Assuming video_df is your DataFrame and it's already loaded
video_df = video_df.dropna(subset=['audio_path'])
max_workers = os.cpu_count() // 2 -3  # Example: half of your CPU cores
print(max_workers)
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    executor.map(run_spleeter, video_df['audio_path'])

8
1


Process SpawnProcess-10:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'run_spleeter' on <module '__main__' (built-in)>


BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

## Audio splitting

In [6]:

import wave

audio_file_path = "/Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals.wav"
# Times to split at, in seconds
split_times = [66, 130, 269, 400, 480, 588]
split_frames = []

# Function to save the split audio
def save_split_audio(audio_data, start_frame, end_frame, sample_width, frame_rate, n_channels, split_index):
    output_file_path = f'//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_{split_index}.wav'
    with wave.open(output_file_path, 'w') as output_file:
        output_file.setnchannels(n_channels)
        output_file.setsampwidth(sample_width)
        output_file.setframerate(frame_rate)
        output_file.writeframes(audio_data[start_frame:end_frame])
    return output_file_path

# Read the audio file and create splits
with wave.open(audio_file_path, 'rb') as wav_file:
    n_frames = wav_file.getnframes()
    sample_width = wav_file.getsampwidth()
    frame_rate = wav_file.getframerate()
    n_channels = wav_file.getnchannels()
    
    # Calculate frame indices for the split times
    split_frames = [int(time * frame_rate) for time in split_times]
    
    # Add the end of the audio file as the last split
    split_frames.append(n_frames)
    
    # Initialize start frame for the first split
    start_frame = 0
    
    # Split the audio and save each part
    audio_splits_paths = []
    for i, end_frame in enumerate(split_frames):
        # Read the frames for the current split
        wav_file.setpos(start_frame)
        audio_data = wav_file.readframes(end_frame - start_frame)
        
        # Save the current split audio
        split_path = save_split_audio(audio_data, 0, end_frame - start_frame, sample_width, frame_rate, n_channels, i)
        audio_splits_paths.append(split_path)
        
        # Set the start frame for the next split
        start_frame = end_frame

audio_splits_paths


['//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_0.wav',
 '//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_1.wav',
 '//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_2.wav',
 '//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_3.wav',
 '//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_4.wav',
 '//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_5.wav',
 '//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_6.wav']

In [ ]:
import SceneNet.Audio as Audio
audio_file_tuple=[('//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/vocals_0.wav','//Users/markrademaker/Downloads/Work/Scriptie/Data/Survivor_full_audio/accompaniment_0.wav')]
embedder = MediaEmbedder()
chunk_duration = 1  # Duration of each chunk in seconds

embeddings_list = []
labels_list = []

for audio_paths in audio_file_tuple:
    embeddings, labels = embedder.embed_audio(audio_paths, chunk_duration, resample_rate=16000)
    embeddings_list.append(embeddings)
    labels_list.append(labels)

# Assuming embeddings_list is a list of tensors and labels_list is a list of labels
embeddings_tensor = torch.tensor(embeddings_list, dtype=torch.float32)
labels_tensor = torch.tensor(labels_list, dtype=torch.long)  # Use torch.long for integer labels

embedding_dataset = EmbeddingsDataset(embeddings_tensor, labels_tensor)
train_loader = DataLoader(embedding_dataset, batch_size=32, shuffle=False)

# Define the biGRU model
audio_model = AudioBiGRU(input_size=embeddings_tensor.size(-1), hidden_size=128, num_layers=2, num_classes=2)
trainer = AudioModelTrainer(audio_model, train_loader, learning_rate=0.001, num_epochs=10)
trainer.train()

## Comnmannd line training

## YAMnet

# VGG16

In [ ]:
vgg16_classifier = VGG16Classifier()
img_path = 'path_to_your_image.jpg'
vgg_features = vgg16_classifier.get_features_before_last_layer(img_path)
vgg_predictions = vgg16_classifier.predict(img_path)

## Recurrent Cells


In [ ]:
# Assuming labels and embeddings are defined
scene_labels = [0,1]  # Replace with your scene labels
# Create a combined dataset and data loader
class CombinedDataset(Dataset):
    def __init__(self, audio_embeddings, video_embeddings, labels):
        self.audio_embeddings = audio_embeddings
        self.video_embeddings = video_embeddings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.audio_embeddings[idx], self.video_embeddings[idx], self.labels[idx]

combined_dataset = CombinedDataset(audio_embeddings, video_embeddings, scene_labels)
combined_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)

# Create an IntegratedModelTrainer instance
trainer = IntegratedModelTrainer(audio_model, video_model, mlp_model, combined_loader)

# Train the integrated model
trainer.train()

## Fine-Tuning

In [ ]:
# Later, for fine-tuning only the MLP
trainer.toggle_biGRU_trainability(trainable=False)
trainer.train()